In [ ]:
import numpy as np
import pandas as pd
import gzip

In [ ]:
#loading images
train=gzip.GzipFile('/content/drive/MyDrive/RFMID/train.npy.gz', "r")
train_images=np.load(train)
test=gzip.GzipFile('/content/drive/MyDrive/RFMID/test.npy.gz', "r")
test_images=np.load(test)
# val=gzip.GzipFile('/content/drive/MyDrive/RFMID/val.npy.gz', "r")
# val_images=np.load(val)
print(train_images.shape)
print(test_images.shape)
# print(val_images.shape)

(1519, 400, 400, 3)
(506, 400, 400, 3)


In [ ]:
train_label = pd.read_csv('/content/drive/MyDrive/RFMID/a. RFMiD_Training_Labels_final.csv')    # reading the csv file
print(train_label.shape)
train_label.drop(['Disease_Risk','ID'],axis=1,inplace=True)
print(train_label.shape)
y_train=np.array(train_label)
print(y_train.shape)

(1519, 9)
(1519, 7)
(1519, 7)


In [ ]:
test_label = pd.read_csv('/content/drive/MyDrive/RFMID/c. RFMiD_Testing_Labels_final.csv')    # reading the csv file
print(test_label.shape)
test_label.drop(['Disease_Risk','ID'],axis=1,inplace=True)
print(test_label.shape)
y_test=np.array(test_label)
print(y_test.shape)

(506, 9)
(506, 7)
(506, 7)


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(400, 400, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
for layer in VGG_model.layers:
	layer.trainable = False

In [ ]:
VGG_model.save("/content/drive/MyDrive/RFMID/vgg_final.h5")

In [ ]:
feature_extractor=VGG_model.predict(train_images)

In [ ]:
f = gzip.GzipFile("/content/drive/MyDrive/RFMID/features_final.npy.gz", "w")
np.save(file=f, arr=feature_extractor)
f.close()

In [ ]:
f = gzip.GzipFile('/content/drive/MyDrive/RFMID/features_final.npy.gz', "r")
train_images=np.load(f)
train_images.shape

(1519, 12, 12, 512)

In [ ]:
features = train_images.reshape(train_images.shape[0], -1)

In [ ]:
X_for_training = features
X_for_training.shape

(1519, 73728)

In [ ]:
y_train.shape

(1519, 7)

In [ ]:
# logistic regression for multi-class classification using a one-vs-rest
from sklearn.datasets import make_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
ovr = OneVsRestClassifier(model)
# fit model
ovr.fit(X_for_training, y_train)
# make predictions
#yhat = ovr.predict(X_test)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                            

In [ ]:
import pickle
file = open('/content/drive/MyDrive/RFMID/final_model.pkl', 'wb')
pickle.dump(ovr,file)

In [ ]:
import pickle
file = open('/content/drive/MyDrive/RFMID/final_model.pkl', 'rb')
ovr = pickle.load(file)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
img=image.load_img("/content/955.png",target_size=(400,400))

img=image.img_to_array(img)

X=np.expand_dims(img, axis=0)
X.shape

(1, 400, 400, 3)

In [ ]:
test=VGG_model.predict(X)
test.shape

(1, 12, 12, 512)

In [ ]:
val = test.reshape(test.shape[0], -1)
val.shape

(1, 73728)

In [ ]:
pred=ovr.predict(val)
print(pred)

[[0 1 1 0 1 1 0]]


In [ ]:
[[0 1 1 1 0 0 0]]
[[0 1 0 1 1 0 0]]
[[1 1 1 1 1 0 0]]
[[0 1 1 0 1 1 0]]